In [2]:
import pandas as pd
import numpy as np

# ✅ Set random seed and initialize configuration
np.random.seed(42)
num_seasons = 10
season_length = 130

# ✅ Fixed Parameters
sales_tax_rate = 0.055
head_tax_per_passenger = 10
waste_cost_per_visitor = 2.25
conservation_cost_per_visitor = 3
cruise_profit_margin = 0.175
B_tour = 0.60

minimum_daily_cruise_passengers = 5000
maximum_daily_cruise_passengers = 15000
minimum_daily_independent_tourist = 100
maximum_daily_independent_tourists = 1000

# ✅ Environmental Calculations
Sm_Ship_Mult = 0.4
NoX_Large = 199283.8356
NoX_Small = NoX_Large * Sm_Ship_Mult

# ✅ Tourist Types
tourist_types = {
    'budget': {'spending_factor': 0.8, 'impact_factor': 1.2, 'satisfaction_effect': -0.001},
    'average': {'spending_factor': 1.0, 'impact_factor': 1.0, 'satisfaction_effect': -0.002},
    'luxury': {'spending_factor': 1.3, 'impact_factor': 1.5, 'satisfaction_effect': -0.003},
    'eco_friendly': {'spending_factor': 0.9, 'impact_factor': 0.5, 'satisfaction_effect': 0.001}
}
type_distribution = {
    'budget': 0.25,
    'average': 0.4,
    'luxury': 0.2,
    'eco_friendly': 0.15
}

# ✅ Output Storage
combined_output = []

# ✅ Ships function (from original)
def ships(x):
    if x > 16000:
        return 4, 1
    elif 14000 < x <= 16000:
        return 4,0
    elif 13000 < x <= 14000:
        return 3,2
    elif 12000 < x <= 13000:
        return 3,1
    elif 10000 < x <= 12000:
        return 3,0
    elif 9000 < x <= 10000:
        return 2,2
    elif 8000 < x <= 9000:
        return 2,1
    elif 6000 < x <= 8000:
        return 2,0
    elif 5000 < x <= 6000:
        return 1,2
    elif 4000 < x <= 5000:
        return 1,1
    elif x < 4000:
        return 1, 0

# ✅ Run Simulation Across Seasons
for season in range(1, num_seasons + 1):
    season_cbj = 0
    season_local = 0
    season_cruise = 0
    season_impact = 0
    satisfaction = 100
    satisfaction_log = []

    for day in range(season_length):
        # Simulate daily tourist counts
        cruise_passengers = int(np.random.normal(10000, 2000))
        cruise_passengers = np.clip(cruise_passengers, 5000, 15000)
        independent_tourists = int(np.random.normal(500, 150))
        independent_tourists = np.clip(independent_tourists, 100, 1000)
        total_visitors = cruise_passengers + independent_tourists

        # Base financials and operations
        spending_cruise = np.random.uniform(180, 250)
        spending_ind = np.random.uniform(250, 400)
        cruise_ticket_price = np.random.randint(1200, 1800)
        cruise_operating_cost = np.random.randint(100, 150)
        infrastructure_cost = np.random.randint(10000, 30000)

        # Revenue & Impact with Tourist Types
        total_spend = 0
        total_impact = 0
        satisfaction_change = 0

        for t_type, share in type_distribution.items():
            count = int(total_visitors * share)
            spending = np.random.uniform(180, 400, count) * tourist_types[t_type]['spending_factor']
            impact = tourist_types[t_type]['impact_factor'] * count
            satisfaction_change += tourist_types[t_type]['satisfaction_effect'] * count
            total_spend += np.sum(spending)
            total_impact += impact

        sales_tax = total_spend * sales_tax_rate
        head_tax = cruise_passengers * head_tax_per_passenger
        waste_cost = total_visitors * waste_cost_per_visitor
        conservation_cost = total_visitors * conservation_cost_per_visitor

        cbj_profit = sales_tax + head_tax - waste_cost - conservation_cost - infrastructure_cost
        local_profit = total_spend * B_tour
        cruise_revenue = cruise_passengers * cruise_ticket_price
        cruise_cost = cruise_passengers * cruise_operating_cost
        cruise_profit = (cruise_revenue - cruise_cost) * cruise_profit_margin

        Ships_Large, Ships_Small = ships(cruise_passengers)
        NOx = NoX_Large * Ships_Large + NoX_Small * Ships_Small

        # Track seasonal accumulation
        season_cbj += cbj_profit
        season_local += local_profit
        season_cruise += cruise_profit
        season_impact += total_impact
        satisfaction += satisfaction_change
        satisfaction_log.append(satisfaction)

    combined_output.append({
        "Season": season,
        "CBJ_Profit_Millions": season_cbj / 1_000_000,
        "Local_Business_Profit_Millions": season_local / 1_000_000,
        "Cruise_Line_Profit_Millions": season_cruise / 1_000_000,
        "Total_Environmental_Impact": season_impact,
        "Final_Community_Satisfaction": satisfaction,
        "Average_Community_Satisfaction": np.mean(satisfaction_log),
        "Avg_Ships_Large": Ships_Large,
        "Avg_Ships_Small": Ships_Small,
        "NOx_Last_Day": NOx
    })

# ✅ Output to Excel
df = pd.DataFrame(combined_output)
output_path = "Advanced_Full_Simulation_With_All_Logic.xlsx"
df.to_excel(output_path, index=False)

output_path


'Advanced_Full_Simulation_With_All_Logic.xlsx'